In [1]:
import time
from tqdm import tqdm

import torch

from torchani.datasets import COMP6v1
import torchani
from torchani.units import hartree2kcalmol

In [2]:
ds = COMP6v1('/home/nick/Datasets/comp6v1/')
device = torch.device('cuda')
ani2x = torchani.models.ANI2x().to(device)

Checking integrity of files for dataset COMP6v1:   0%|          | 0/1 [00:00<?, ?it/s]

Name: COMP6-v1_full-B973c-def2mTZVP
ANI h5py store:
{'Conformer groups': 97,
 'Conformers': '99,968',
 'Properties': ['D3.energy-corrections',
                'D3.force-corrections',
                '_id',
                'coordinates',
                'dipole',
                'energies',
                'forces',
                'quadrupole',
                'species'],
 'Store Metadata': {}}


In [9]:
_start = time.perf_counter()
with ds.keep_open("r") as rods:
    pbar = tqdm(total=ds.num_conformers)
    for group, j, conformer in rods.chunked_items(max_size=2500):
        species = conformer["species"].to(device)
        coordinates = conformer["coordinates"].to(device)
        coordinates.requires_grad_(True)
        print(coordinates.shape)
        energies = ani2x((species, coordinates)).energies
        print(energies.shape)
        print(energies.sum().shape)
        forces = -torch.autograd.grad(energies.sum(), coordinates)[0]
        print(forces.shape)
        pbar.update(len(coordinates))
        break
    pbar.close()
print(f"Time elapsed: {time.perf_counter() - _start} s")
# This took about 7.3 min in CPU of my laptop
# Intel(R) Core(TM) i5-10300H CPU @ 2.50GHz
# My laptop's GPU doesn't have enough memory to do it

  0%|          | 128/101352 [00:00<00:50, 2016.12it/s]

torch.Size([128, 20, 3])
torch.Size([128])
torch.Size([])
torch.Size([128, 20, 3])
Time elapsed: 0.0727061778306961 s
